### Question 1:

For each point in the complex plane $c = x + iy$, with $-2 < x < 2$ and $-2 < y < 2$, set $z_0 = 0$ and iterate the equation $z_{i + 1} = z_i^2 + c$. 
Note what happens to the $z_i$'s: some points will remain bounded in absolute value $|z|^2 = \Re(z)^2 + \Im(z)^2$, while others will run off to infinity. 
Make an image  in which your points $c$ that diverge are given one color and those that stay bounded are given another.
Make a second image where the points are coloured by a colourscale that indicates the iteration number at which the given point diverged.

For this question, put the code that does the iteration in a function and place this function in a separate .py file which you import in your .ipynb.
Perform the plots in the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from iter_function import iter_z

In [ ]:
x = np.linspace(-2, 2, 500)
y = np.linspace(-2, 2, 500)

X, Y = np.meshgrid(x, y)
C = X + 1j*Y

In [ ]:
vec_iter_z = np.vectorize(iter_z)
escape_times = vec_iter_z(C)

fig, axs = plt.subplots(1, 2, figsize=(14, 6))

binary_mask = (escape_times < 100).astype(int)
im1 = axs[0].imshow(binary_mask, extent=(-2, 2, -2, 2), cmap='gist_stern_r')
axs[0].set_title('Divergence (Binary)', fontsize=14)
axs[0].set_xlabel('Re')
axs[0].set_ylabel('Im')
bounded_patch = mpatches.Patch(color=plt.cm.gist_stern_r(0.0), label='Bounded (0)')
diverged_patch = mpatches.Patch(color=plt.cm.gist_stern_r(1.0), label='Diverged (1)')
axs[0].legend(handles=[bounded_patch, diverged_patch], loc='upper right')

im2 = axs[1].imshow(escape_times, extent=(-2, 2, -2, 2), cmap='gist_stern')
cbar = fig.colorbar(im2, ax=axs[1], label='Iteration')
axs[1].set_title('Divergence Time Coloring', fontsize=14)
axs[1].set_xlabel('Re')
axs[1].set_ylabel('Im')
plt.tight_layout()
plt.show()


### Question 2:

One of the earliest demonstrations that deterministic physical systems could exhibit unpredictable behavior was given by Edward Lornez, a meteorologist. The original paper is https://journals.ametsoc.org/view/journals/atsc/20/2/1520-0469_1963_020_0130_dnf_2_0_co_2.xml?tab_body=pdf, which is worth downloading and looking over.

Lorenz was interested in modeling the behavior of Earth's atmosphere, i.e., a thin atmosphere (thin relative to the radius of Earth) heated from below (the air is heated by infrared radiation from the ground, or by condensing water vapor in thunder clouds, rather than by sunlight). Lorenz applies a Fourier transform to the basic equations, and truncates the number of Fourier modes, keeping only three, with amplitudes denoted by $W\equiv(X, Y, Z)$.

The equations (Lorenz' equations 25, 26, and 27) are

\begin{eqnarray}
\dot X &=& -\sigma(X-Y)\\
\dot Y &=& rX -Y - XZ\\
\dot Z &=& -bZ + XY
\end{eqnarray}

The three dimensionless parameters are $\sigma$, the Prandtl number (the ratio of the kinematic viscosity to the thermal diffusivity), the Rayleigh number $r$ (which depends on the vertical temperature difference between the top and bottom of the atmosphere), and b, which is a dimensionless length scale.

Note that there are non-linear terms in the second and third equations;
these terms result in very complex dynamics.

Your task is to:
1. code up the equations, using a function definition, with a proper docstrings (inside triple quotes)

2. use an ode solver of your choice, i.e., solve_ivp, or ode, to integrate the equations for t=60 (in dimensionless time units). Use Lorenz' initial conditions $W_0=[0., 1., 0.]$ and his parameter values [$\sigma, r, b$] = [10., 28, 8./3.].

3. Reproduce Lorenz' Figure 1. Label both axes! Note that Lorenz uses $N=t/\Delta t$ to label his plots (here $\Delta t=0.01$).

4. Reproduce Lorenz' Figure 2. You will likely have to ask for output at very closely spaced time intervals, e.g., if you use solve_ivp, you will need something like t = np.linspace(14, 19, 1000) followed by W = sol.sol(t). Again, label both axes.

5. Now find the solution using the same values of $(\sigma, r, b)$, but this time with initial conditions very slightly different than $W_0$, say $W'_0 = W_0+[0., 1.e-8, 0] = [0., 1.00000001, 0.]$; note that adding the two lists (as indicated here) will not work, so you should google to find out how to add two lists element by element. Calculate the distance between $W'$ and $W$ as a function of time, and plot the result on a semilog plot (linear time, log distance). A straight line on such a plot, which is what Lorenz found, indicates exponential growth. Thus a small error in the initial condition will grow rapidly, meaning that predictions of future behavior will not be accurate.


For this question you may put all of your code in the jupyter notebook or place some of it in a separate .py file as you see fit. The only requirement is that you make sure it runs correctly in the correct order (ie restart your kernel and run each cell in order to check it works correctly before submitting).


In [ ]:
from scipy.integrate import solve_ivp

const = {'sigma':10.,
       'r':28,
       'b':8./3.}

W0 = np.array([0.,1.,0.])

def dx(X, Y, Z):
    return -const['sigma'] * (X - Y)

def dy(X, Y, Z):
    return const['r'] * X - Y - X * Z

def dz(X, Y, Z):
    return -const['b'] * Z + X * Y

def odes(t, W):
    """
    Computes the derivatives of the Lorenz system at a given time t and state W.

    Parameters
    ----------
    t : float
        Current time (not used explicitly as the system is autonomous, but required by ODE solvers).
    W : list or array-like of float
        The current state of the system, [X, Y, Z].

    Returns
    -------
    list of float
        The derivatives [dX/dt, dY/dt, dZ/dt] at time t.
    """
    X, Y, Z = W
    return [dx(X, Y, Z), dy(X, Y, Z), dz(X, Y, Z)]


In [ ]:
sol = solve_ivp(odes, (0, 60), W0)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10, 8))

mask1 = sol.t * 100 <= 1000
axs[0].plot(sol.t[mask1] * 100, sol.y[1][mask1], color='black', linewidth=1.2)
axs[0].plot(sol.t[mask1] * 100, sol.t[mask1] * 0, color='black', linewidth=1.2)
axs[0].set_ylabel("Y", fontsize=12)

mask2 = (sol.t * 100 >= 1000) & (sol.t * 100 <= 2000)
axs[1].plot(sol.t[mask2] * 100, sol.y[1][mask2], color='black', linewidth=1.2)
axs[1].plot(sol.t[mask2] * 100, sol.t[mask2] * 0, color='black', linewidth=1.2)
axs[1].set_ylabel("Y", fontsize=12)

mask3 = (sol.t * 100 >= 2000) & (sol.t * 100 <= 3000)
axs[2].plot(sol.t[mask3] * 100, sol.y[1][mask3], color='black', linewidth=1.2)
axs[2].plot(sol.t[mask3] * 100, sol.t[mask3] * 0, color='black', linewidth=1.2)
axs[2].set_ylabel("Y", fontsize=12)
axs[2].set_xlabel("Iteration", fontsize=14)

axs[0].set_xlim([0, 1000])
axs[1].set_xlim([1000, 2000])
axs[2].set_xlim([2000, 3000])

axs[0].set_xticks(np.linspace(0, 1000, 6))
axs[1].set_xticks(np.linspace(1000, 2000, 6))
axs[2].set_xticks(np.linspace(2000, 3000, 6))

plt.tight_layout()
plt.show()



In [ ]:
t_span = (0, 60)
t_eval = np.linspace(14, 19, 1000)

sol2 = solve_ivp(odes, t_span, W0, t_eval=t_eval)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].plot(sol2.y[1], sol2.y[2], color='black', linewidth=1.2)
axs[0].set_xlabel("Y", fontsize=14)
axs[0].set_ylabel("Z", fontsize=14)
axs[0].set_title("Phase Space Plot:\nY vs Z", fontsize=14)

axs[1].plot(sol2.y[1], sol2.y[0], color='black', linewidth=1.2)
axs[1].invert_yaxis()
axs[1].set_xlabel("Y", fontsize=14)
axs[1].set_ylabel("X", fontsize=14)
axs[1].set_title("Phase Space Plot:\nY vs X (Y-axis Flipped)", fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
W0_del = np.array([0,1e-8,0])
W2 = W0 + W0_del

sol_3 = solve_ivp(odes, (0, 60), W0)
sol_4 = solve_ivp(odes, (0, 60), W2)

diff = np.linalg.norm((sol_3.y - sol_4.y), axis=0)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(sol_4.t, diff, color='black', linewidth=1.5)
plt.yscale('log')
plt.xlabel('Time (t)', fontsize=14)
plt.ylabel(r'Distance $\|W - W^\prime\|$ (log scale)', fontsize=14)
plt.title('Exponential Growth of Initial Condition Perturbation', fontsize=16)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()
